In [7]:
# -*- coding: utf-8 -*-
"""
Alpaca API - Calculate and store MACD TI using historical data (V2 API)

"""

import os
import json
import alpaca_trade_api as tradeapi
from alpaca_trade_api.rest import REST, TimeFrame



API_KEY = os.getenv("ALPACA_API_KEY")
API_SECRET = os.getenv("ALPACA_SECRET_KEY")

endpoint = "https://paper-api.alpaca.markets"

headers = {'APCA-API-KEY-ID': API_KEY,
           'APCA-API-SECRET-KEY': API_SECRET}

def hist_data(symbols, start_date ="2021-12-01", timeframe="Minute"):
    """
    returns historical bar data for a list of tickers e.g. symbols = ["MSFT,AMZN,GOOG"]
    """
    df_data = {}
    api = REST(headers["APCA-API-KEY-ID"], headers["APCA-API-SECRET-KEY"], base_url=endpoint)
    for ticker in symbols:
        if timeframe == "Minute":
            df_data[ticker] = api.get_bars(ticker, TimeFrame.Minute, start_date, adjustment='all').df
        elif timeframe == "Hour":
            df_data[ticker] = api.get_bars(ticker, TimeFrame.Hour, start_date, adjustment='all').df
        else:
            df_data[ticker] = api.get_bars(ticker, TimeFrame.Day, start_date, adjustment='all').df
    return df_data
        
data_dump = hist_data(["AMZN"], start_date ="2022-09-01", timeframe="Hour") 
print(data_dump)

def MACD(df_dict, a=12 ,b=26, c=9):
    '''function to calculate MACD
       typical values a(fast moving average) = 12; 
                      b(slow moving average) =26; 
                      c(signal line ma window) =9'''
    for df in df_dict:
        df_dict[df]["ma_fast"] = df_dict[df]["close"].ewm(span=a, min_periods=a).mean()
        df_dict[df]["ma_slow"] = df_dict[df]["close"].ewm(span=b, min_periods=b).mean()
        df_dict[df]["macd"] = df_dict[df]["ma_fast"] - df_dict[df]["ma_slow"]
        df_dict[df]["signal"] = df_dict[df]["macd"].ewm(span=c, min_periods=c).mean()
        df_dict[df].drop(["ma_fast","ma_slow"], axis=1, inplace=True)

MACD(data_dump)

      

{'AMZN':                              open    high     low     close   volume  \
timestamp                                                              
2022-09-01 08:00:00+00:00  125.50  125.76  125.20  125.5000    31958   
2022-09-01 09:00:00+00:00  125.60  125.77  125.37  125.7200     9712   
2022-09-01 10:00:00+00:00  125.59  125.77  125.41  125.7200    13359   
2022-09-01 11:00:00+00:00  125.58  126.46  125.42  126.3000   251065   
2022-09-01 12:00:00+00:00  126.10  126.77  125.41  125.8100   276230   
...                           ...     ...     ...       ...      ...   
2022-09-12 13:00:00+00:00  134.15  135.47  133.95  135.3828  7259267   
2022-09-12 14:00:00+00:00  135.38  136.37  135.05  135.1850  9054521   
2022-09-12 15:00:00+00:00  135.18  136.02  134.82  135.2699  6163088   
2022-09-12 16:00:00+00:00  135.27  135.59  134.85  135.3500  4677800   
2022-09-12 17:00:00+00:00  135.36  135.70  134.89  135.5000  1652878   

                           trade_count        vwap  
t